## Derivation of Required Power

McLean uses a simple equation to calculate the power needed for level flight. We will derive this equation here.

We start with the definitions of the lift, drag, and moment coefficients are:

| $C_l = \frac{L}{qS}$ | $C_d = \frac{D}{qS}$| $C_m = \frac{M}{qSc}$ |

Where
- $L$ is the lift force
- $D$ is the drag force
- $M$ is the moment about the 
- $q$ is the dymanic pressure
- $S$ is a reference area
- $c$ is the chord of the airfoil

By convention, the pitching moment positive when it pitches the airfoil in the nose-up direction. For cambered airfoils, a lift force acting at the aerodynamic center will pitch nose-down so the pitching moment coefficient will be negative.

The definition of dynamic pressure is:

$$
q = \frac{1}{2}\rho u^2
$$

![Pitchin Moment](./assets/images/pitching-moment.png)

The aerodynamic forces act through the *Mean Aerodynamic center*, so we need to figure out the forces acting on this model.

Let $S_w$ be the surface area of the wing, and $S_s$ be the surface area of the stabilizer. The total moments about the center of gravidy can be calculated as follows:

$$
x L_w - l L_s
$$

Where:

$$
L_w = q S_w C_{lw}
$$

and

$$
L_s = q S_s C_{ls}
$$


In [1]:
import sympy

In [16]:
c_l, c_d, rho, u, S = sympy.symbols('c_l c_d rho u S')

In [17]:
E1 = c_l - rho * u ** 2/2

In [18]:
E1

c_l - rho*u**2/2

In [19]:
M_w, M_ac = sympy.symbols('M_w M_ac')

In [20]:
M_ac

M_ac

In [63]:
S_f = 30.0
S_r = 0.4 * S_f
C_f = 0.25 * 2
C_r = 0.25 * 1.5
C_mf = 0.10
C_mr = 0.05
x = .50 * 2
l = 10

C_lr = (-(x/C_f) - C_mf - C_mr*(S_r*C_r/(S_f*C_f))) / \
       ((1 + x/l)*(S_r * l / (S_f*C_f)))

C_lr

-0.2403409090909091

In [41]:
C_l = (1 + S_r*C_lr/S_f) / \
      (1 + S_r/S_f)
C_l

0.6456168831168831

Let's calculate the flight speed using Walt's formula:

In [46]:
import pint
u = pint.UnitRegistry()

In [62]:
WT = (1.2 + 0.86) * u.grams
AW = 30.0 * u.inches ** 2
PCW = 0.4
S = (1 + PCW) * AW
B_w = 15
B_s = 9
C_f = AW / B_w
C_r = S / B_s

In [48]:
density = 0.00119 * 2 * u.slugs/u.ft**3 # air density from program

In [49]:
k1 = (WT * 2 * u.gravity) / (density * AW)
k2 = C_l
VE = (k1/k2)**0.5

In [50]:
VE.to('ft/sec')

5.326726989562554 <Unit('foot / second')>

This seems a bit fast for his A6

Now, lets use the formula from McLean's paper:

In [61]:
import math

V = ((WT * u.gravity /(C_l* S))/ density)**0.5
V.to('ft/sec')

3.183328246019416 <Unit('foot / second')>

The coefficient data used in McLean's paper do not work using Erbach's scheme. Time for some adjustments.

In [70]:
S_f = 30.0 * u.inches**2
S_r = 0.4 * S_f
C_f = 0.25 * 2 * u.inch
C_r = 0.25 * 1.5 * u.inch
C_mf = 0.10
C_mr = 0.05
C_la = 0.135
C_da = 0.009
x = .50 * 2 * u.inch
l = 10 * u.inch
M_f = C_la * 

C_lr = (-(x/C_f) - C_mf - C_mr*(S_r*C_r/(S_f*C_f))) / \
       ((1 + x/l)*(S_r * l / (S_f*C_f)))

C_lr

-0.2403409090909091 <Unit('dimensionless')>

In [71]:
k1 = (WT * 2 * u.gravity) / (density * (S_f + S_r))
k2 = C_l
VE = (k1/k2)**0.5

In [73]:
VE.to('ft/sec')

4.501905979006015 <Unit('foot / second')>

This is better. Perhaps those moment values are off!

I found some wind tunnel data data for a 3% circular arc airfoil that will be useful. This chart is from an article titled *Effectiveness of Large-Camber Circular Arc Airfoil
at Very Low Reynolds Numbers*. McLean assumed coefficient values that are not in line with this data, so we will update the coefficients.

![Circular Arc Coefficients](./assets/images/3%-circular-arc.png)

![Circular Arc Coefficients](./assets/images/3%-circular-arc-CM.png)

Our test data from Erbach's code has the wing at +2 degrees, and the stab at -2 degrees. Here are the coefficients we will use in our next test (using eyes only!)

In [77]:
C_lw = 0.35
C_dw = 0.05
C_mw = -0.08

C_ls = -0.20
C_ds = 0.05
C_ms = -0.04

In [78]:
C_lr = (-(x/C_f) + C_mw + C_ms*(S_r*C_r/(S_f*C_f))) / \
       ((1 + x/l)*(S_r * l / (S_f*C_f)))

C_lr

-0.2377272727272727 <Unit('dimensionless')>

In [79]:
C_l = (1 + S_r*C_ls/S_f) / \
      (1 + S_r/S_f)
C_l

0.6571428571428571 <Unit('dimensionless')>

In [80]:
k1 = (WT * 2 * u.gravity) / (density * (S_f + S_r))
k2 = C_l
VE = (k1/k2)**0.5

In [82]:
VE.to('ft/sec')

4.462250678203397 <Unit('foot / second')>